<a href="https://colab.research.google.com/github/rahmaoktafer/Penny_Path/blob/main/Model_ML%20Recap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load the dataset
data = pd.read_excel("student_spending.xlsx")

# Select the first two users
first_user = data.iloc[0]  # First user
second_user = data.iloc[1]  # Second user

# Define spending categories
spending = ['tuition','housing', 'food', 'transportation', 'books_supplies',
            'entertainment', 'personal_care', 'technology', 'health_wellness', 'miscellaneous']

# Define income and specific categories to be applied on the 1st day
income = ['monthly_income','financial_aid']

# Function to generate daily spending for a given user and month
def generate_daily_spending(user, start_date, days_in_month):
    daily_spending = []

    for day in range(days_in_month):
        daily_row = {
            'index': user['index'],  # Use the user's ID
            'date': start_date + timedelta(days=day),  # Calculate the date
        }

        # For income and fixed expenses (tuition and housing), add the full value only on the 1st day
        if day == 0:
            for category in income:
                daily_row[category] = user[category]
        else:
            for category in income:
                daily_row[category] = 0  # No income or fixed expenses after the first day

        # Add spending data for each category, with randomness
        for category in spending:
            daily_row[category] = np.round(user[category] / days_in_month + np.random.uniform(-1, 1), 2)

        # Append the daily data for this user
        daily_spending.append(daily_row)

    return pd.DataFrame(daily_spending)

# Generate daily spending for the first user (October 2024, 31 days)
october_start_date = datetime(2024, 10, 1)
october_spending = generate_daily_spending(first_user, october_start_date, 31)

# Generate daily spending for the second user (November 2024, 30 days)
november_start_date = datetime(2024, 11, 1)
november_spending = generate_daily_spending(second_user, november_start_date, 30)

# Combine both months into a single DataFrame
combined_spending = pd.concat([october_spending, november_spending], ignore_index=True)

# Save the DataFrame to an Excel file
combined_spending.to_excel("daily_spending_with_income.xlsx", index=False)

print("Daily spending for October and November 2024 has been saved to 'daily_spending_with_income.xlsx'.")


Daily spending for October and November 2024 has been saved to 'daily_spending_with_income.xlsx'.


In [8]:
# Load the daily spending dataset
daily_spending = pd.read_excel("daily_spending_with_income.xlsx")

today = datetime.now()
# Initialize a dictionary to store end-of-month recaps
end_of_month_recaps = {}

# Loop through the months in the dataset
for month in daily_spending['date'].dt.month.unique():
    # Filter data for the specific month
    month_data = daily_spending[daily_spending['date'].dt.month == month]
    year = month_data['date'].dt.year.unique()[0]

    # Calculate total spending and income for the month
    total_spending = month_data[spending].sum()
    total_income = month_data[income].sum()

    # Calculate overall totals
    overall_total_spending = total_spending.sum()
    overall_total_income = total_income.sum()

    # Calculate percentages for each spending category
    spending_percentages = (total_spending / overall_total_spending * 100).round(2) if overall_total_spending > 0 else None

    # Store the recap in the dictionary
    end_of_month_recaps[f"{year}-{month:02d}"] = {
        "spending_recap": total_spending.to_dict(),
        "spending_percentages": spending_percentages.to_dict() if spending_percentages is not None else None,
        "income_recap": total_income.to_dict(),
        "overall_total_spending": overall_total_spending,
        "overall_total_income": overall_total_income,
    }

# Display recaps for previous months and the current month until today
for month_key, recap in end_of_month_recaps.items():
    year, month = map(int, month_key.split("-"))

    if month < today.month or year < today.year:
        # Display previous month recap
        print(f"\nRecap for {month_key}: (Previous Month)")
        print(f"Spending Recap: {recap['spending_recap']}")
        print(f"Spending Percentages: {recap['spending_percentages']}")
        print(f"Income Recap: {recap['income_recap']}")
        print(f"Overall Total Spending: {recap['overall_total_spending']}")
        print(f"Overall Total Income: {recap['overall_total_income']}")
    elif month == today.month and year == today.year:
        # Calculate recap for the current month until today
        current_month_data = daily_spending[
            (daily_spending['date'] <= today) &
            (daily_spending['date'].dt.month == today.month) &
            (daily_spending['date'].dt.year == today.year)
        ]

        current_spending = current_month_data[spending].sum()
        current_income = current_month_data[income].sum()
        current_total_spending = current_spending.sum()
        current_total_income = current_income.sum()

        current_spending_percentages = (
            (current_spending / current_total_spending * 100).round(2) if current_total_spending > 0 else None
        )

        print(f"\nRecap for {month_key}: (Current Month Until Today)")
        print(f"Spending Recap: {current_spending.to_dict()}")
        print(f"Spending Percentages: {current_spending_percentages.to_dict() if current_spending_percentages is not None else None}")
        print(f"Income Recap: {current_income.to_dict()}")
        print(f"Overall Total Spending: {current_total_spending}")
        print(f"Overall Total Income: {current_total_income}")



Recap for 2024-10: (Previous Month)
Spending Recap: {'tuition': 5934.32, 'housing': 708.5500000000001, 'food': 291.34000000000003, 'transportation': 122.44000000000003, 'books_supplies': 190.01999999999995, 'entertainment': 37.379999999999995, 'personal_care': 80.36999999999999, 'technology': 137.35, 'health_wellness': 133.11999999999998, 'miscellaneous': 71.71999999999998}
Spending Percentages: {'tuition': 77.0, 'housing': 9.19, 'food': 3.78, 'transportation': 1.59, 'books_supplies': 2.47, 'entertainment': 0.49, 'personal_care': 1.04, 'technology': 1.78, 'health_wellness': 1.73, 'miscellaneous': 0.93}
Income Recap: {'monthly_income': 958, 'financial_aid': 270}
Overall Total Spending: 7706.61
Overall Total Income: 1228

Recap for 2024-11: (Current Month Until Today)
Spending Recap: {'tuition': 4583.990000000001, 'housing': 518.9300000000001, 'food': 341.45, 'transportation': 75.16, 'books_supplies': 235.47, 'entertainment': 68.35, 'personal_care': 82.67, 'technology': 211.63, 'health_

In [9]:
# Ensure the 'date' column is in datetime format
daily_spending['date'] = pd.to_datetime(daily_spending['date'])

# Group data by month and sum up the spending and income
# Exclude the 'date' column from the sum operation
monthly_summary = daily_spending.groupby(daily_spending['date'].dt.to_period('M'))[[c for c in daily_spending.columns if c != 'date']].sum()

# Get the percentage change from the previous month
monthly_percentage_change = monthly_summary.pct_change().round(4) * 100  # Convert to percentage

# Generate insights for trends
trend_insights = []
current_month = today.strftime('%Y-%m')  # Get the current month in 'YYYY-MM' format
previous_month = (today.replace(day=1) - timedelta(days=1)).strftime('%Y-%m')  # Get the previous month in 'YYYY-MM'

# Check if both current and previous months exist in the data
if current_month in monthly_summary.index.astype(str) and previous_month in monthly_summary.index.astype(str):
    # Add insights for total spending and total income
    total_spending_change = monthly_percentage_change[spending].sum(axis=1).iloc[-1]
    total_income_change = monthly_percentage_change[income].sum(axis=1).iloc[-1]

    if not pd.isna(total_spending_change):  # Ensure the change is valid
        if total_spending_change > 0:
            trend_insights.append(f"Your total spending increased by {total_spending_change:.2f}% compared to last month.")
        elif total_spending_change < 0:
            trend_insights.append(f"Your total spending decreased by {abs(total_spending_change):.2f}% compared to last month.")

    if not pd.isna(total_income_change):  # Ensure the change is valid
        if total_income_change > 0:
            trend_insights.append(f"Your total income increased by {total_income_change:.2f}% compared to last month.")
        elif total_income_change < 0:
            trend_insights.append(f"Your total income decreased by {abs(total_income_change):.2f}% compared to last month.")

    # Add insights for individual spending categories
    for category in spending:
        if not monthly_percentage_change[category].isna().iloc[-1]:  # Check if percentage change is not NaN
            change = abs(monthly_percentage_change[category].iloc[-1])  # Get absolute percentage change
            if monthly_percentage_change[category].iloc[-1] > 0:
                trend_insights.append(f"Your spending on {category} increased by {change:.2f}% compared to last month.")
            elif monthly_percentage_change[category].iloc[-1] < 0:
                trend_insights.append(f"Your spending on {category} decreased by {change:.2f}% compared to last month.")

    # Add insights for individual income categories
    for category in income:
        if not monthly_percentage_change[category].isna().iloc[-1]:  # Check if percentage change is not NaN
            change = abs(monthly_percentage_change[category].iloc[-1])  # Get absolute percentage change
            if monthly_percentage_change[category].iloc[-1] > 0:
                trend_insights.append(f"Your income from {category} increased by {change:.2f}% compared to last month.")
            elif monthly_percentage_change[category].iloc[-1] < 0:
                trend_insights.append(f"Your income from {category} decreased by {change:.2f}% compared to last month.")

# Display the insights
if trend_insights:
    print("\n**Trend Insights Compared to Last Month:**")
    for insight in trend_insights:
        print(insight)
else:
    print("\nNo trend insights available. Ensure there is data for both the current and previous months.")



**Trend Insights Compared to Last Month:**
Your total spending increased by 153.65% compared to last month.
Your total income increased by 229.08% compared to last month.
Your spending on tuition decreased by 17.23% compared to last month.
Your spending on housing decreased by 21.48% compared to last month.
Your spending on food increased by 25.42% compared to last month.
Your spending on transportation decreased by 33.11% compared to last month.
Your spending on books_supplies increased by 32.14% compared to last month.
Your spending on entertainment increased by 98.23% compared to last month.
Your spending on personal_care increased by 11.37% compared to last month.
Your spending on technology increased by 64.91% compared to last month.
Your spending on health_wellness decreased by 3.13% compared to last month.
Your spending on miscellaneous decreased by 3.47% compared to last month.
Your income from monthly_income increased by 5.01% compared to last month.
Your income from financia

In [10]:
import json
from datetime import datetime, timedelta
import pandas as pd

# Assuming 'daily_spending' DataFrame is already loaded with spending and income columns
daily_spending['date'] = pd.to_datetime(daily_spending['date'])

# Initialize a dictionary to store end-of-month recaps
end_of_month_recaps = {}

# Process monthly summaries
for month in daily_spending['date'].dt.month.unique():
    month_data = daily_spending[daily_spending['date'].dt.month == month]
    year = month_data['date'].dt.year.unique()[0]

    total_spending = month_data[spending].sum()
    total_income = month_data[income].sum()

    overall_total_spending = total_spending.sum()
    overall_total_income = total_income.sum()

    spending_percentages = (total_spending / overall_total_spending * 100).round(2) if overall_total_spending > 0 else None

    # Convert NumPy types to native Python types for JSON serialization
    end_of_month_recaps[f"{year}-{month:02d}"] = {
        "spending_recap": {k: v.item() if isinstance(v, (np.int64, np.float64)) else v for k, v in total_spending.to_dict().items()},
        "spending_percentages": {k: v.item() if isinstance(v, (np.int64, np.float64)) else v for k, v in spending_percentages.to_dict().items()} if spending_percentages is not None else None,
        "income_recap": {k: v.item() if isinstance(v, (np.int64, np.float64)) else v for k, v in total_income.to_dict().items()},
        "overall_total_spending": overall_total_spending.item() if isinstance(overall_total_spending, (np.int64, np.float64)) else overall_total_spending,
        "overall_total_income": overall_total_income.item() if isinstance(overall_total_income, (np.int64, np.float64)) else overall_total_income,
    }

# Generate insights for trends
today = datetime.now()
current_month = today.strftime('%Y-%m')
previous_month = (today.replace(day=1) - timedelta(days=1)).strftime('%Y-%m')

# Summarize the data for JSON
trend_insights = []
monthly_summary = daily_spending.groupby(daily_spending['date'].dt.to_period('M'))[[c for c in daily_spending.columns if c != 'date']].sum()
monthly_percentage_change = monthly_summary.pct_change().round(4) * 100

if current_month in monthly_summary.index.astype(str) and previous_month in monthly_summary.index.astype(str):
    total_spending_change = monthly_percentage_change[spending].sum(axis=1).iloc[-1]
    total_income_change = monthly_percentage_change[income].sum(axis=1).iloc[-1]

    if not pd.isna(total_spending_change):
        if total_spending_change > 0:
            trend_insights.append(f"Your total spending increased by {total_spending_change:.2f}% compared to last month.")
        elif total_spending_change < 0:
            trend_insights.append(f"Your total spending decreased by {abs(total_spending_change):.2f}% compared to last.")
    if not pd.isna(total_income_change):
        if total_income_change > 0:
            trend_insights.append(f"Your total income increased by {total_income_change:.2f}% compared to last month.")
        elif total_income_change < 0:
            trend_insights.append(f"Your total income decreased by {abs(total_income_change):.2f}% compared to last month.")

    for category in spending:
        if not monthly_percentage_change[category].isna().iloc[-1]:
            change = abs(monthly_percentage_change[category].iloc[-1])
            if monthly_percentage_change[category].iloc[-1] > 0:
                trend_insights.append(f"Your spending on {category} increased by {change:.2f}% compared to last month.")
            elif monthly_percentage_change[category].iloc[-1] < 0:
                trend_insights.append(f"Your spending on {category} decreased by {change:.2f}% compared to last month.")

    for category in income:
        if not monthly_percentage_change[category].isna().iloc[-1]:
            change = abs(monthly_percentage_change[category].iloc[-1])
            if monthly_percentage_change[category].iloc[-1] > 0:
                trend_insights.append(f"Your income from {category} increased by {change:.2f}% compared to last month.")
            elif monthly_percentage_change[category].iloc[-1] < 0:
                trend_insights.append(f"Your income from {category} decreased by {change:.2f}% compared to last month.")

# Combine into a single JSON object
output = {
    "monthly_recaps": end_of_month_recaps,
    "trend_insights": trend_insights
}

output_file = "monthly_trend_insights.json"

# Simpan JSON ke file
with open(output_file, "w") as file:
    json.dump(output, file, indent=4)

print(f"JSON data has been saved to {output_file}")


JSON data has been saved to monthly_trend_insights.json
